In [1]:
#Import packages 

import numpy as np
import pandas as pd
import statistics
from scipy import stats

In [2]:
#Import, and slice data (2nd sample group first 100 responses from revised survey, not including 1st sample group of 20)

df = pd.read_csv("UMass Sustainability Initiative.csv", sep=",")
df = df.iloc[0:100]
#df.info()

In [7]:
#Data Cleaning

df.rename(columns = { 'How old are you?':'Age', 'What is your gender?':'Gender', 'What year are you?':'Year', 'What is your major?':'Major',
                     'Do you live on campus?':'On Campus', 'If you do live on campus, where do live?':'Location',
                     'How often do you recycle?':'Frequency', 'Why do you recycle?':'Reasoning',
                     'Are the recycling rules on campus clearly presented?':'Rules', 
                     'If not, if the recycling rules were clearer would you be more likely to recycle?':'Likelihood',
                     'Do you look for the recycle symbol before recycling?':'Symbol', 'What items do you usually recycle?':'Items',
                     'Did the survey overall make you think about your own personal recycling methods and habits?':'Habits',
                     'Whats the best way to reach you for the prize draw? (Email, Phone, etc. )':'Prize'}, inplace = True)

df.fillna('No', inplace=True)

#Splitting strings into arrays
df['Reasoning'] = df['Reasoning'].str.split(";", expand = False)
df['Items'] = df['Items'].str.split(";", expand = False)

#Replacing Majors with relevant groups, by engineers, Computer Science, Business 
df['Major'].replace(dict.fromkeys(['CIS', 'Computer Science', 'Computer science', 'Computer science'], 
                                  'College of Engineering'), inplace = True)
df['Major'].replace(dict.fromkeys(['MIS', 'Accounting', 'Marketing'], 'College of Business'), inplace = True)

df.Age = df.Age.astype(int)

df.head(3)

,Timestamp,Age,Gender,Year,Major,On Campus,Location,Frequency,Reasoning,Rules,Likelihood,Symbol,Items,Habits,Prize
0,2020/02/18 3:03:42 PM EST,20,Male,Sophomore,College of Engineering,Yes,Birch,Often,NaN,Yes,Yes,Yes,NaN,3,No
1,2020/02/18 3:05:49 PM EST,20,Male,Junior,College of Engineering,Yes,Aspen,Always,NaN,No,Yes,No,NaN,4,jegan2@umassd.edu
2,2020/02/18 3:09:10 PM EST,23,Male,Junior,College of Engineering,No,No,Always,NaN,Yes,No,No,NaN,2,No


In [4]:
#basic data analysis for quantitive data, habits and age distribution

def distribution(df):
    print(f'The mean is {df.mean()} the Standard Erorr of the mean is {stats.sem(df, axis=None, ddof=0)}')
    print(f'The variance is {statistics.variance(df)} the standard deviation is {df.std()}')
    print(f'The min is {df.min()}, Q1 is {np.quantile(df, .25)}, median is {df.median()}, Q3 is {np.quantile(df, .75)}, max is {df.max()}')
    print(f'Skewness: {stats.skew(df)}')
        
distribution(df['Habits'])


The mean is 3.67 the Standard Erorr of the mean is 0.11580587204455565
The variance is 1.3546464646464647 the standard deviation is 1.1638928063384812
The min is 1, Q1 is 3.0, median is 4.0, Q3 is 5.0, max is 5
Skewness: -0.8025214276103186


In [5]:
#basic data analysis for qualtitve data, Major Location Frequency Reasoning Items ... etc

#to be filled in...

In [6]:
#Sorting entries (potential group of interest could be people with low habits, majority of people had high schore for habits)

df.sort_values('Major', ascending=True)

,Timestamp,Age,Gender,Year,Major,On Campus,Location,Frequency,Reasoning,Rules,Likelihood,Symbol,Items,Habits,Prize
94,2020/02/18 4:11:04 PM EST,21,Male,Junior,Accounting,No,No,Sometimes,[Ease of Access],I didn't know there were any recycling rules!,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Plastic Ba...",5,7742221673
49,2020/02/18 3:44:21 PM EST,18,Male,Freshman,Biochem,Yes,Maple,Often,[Sustainability],No,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Plastic Ba...",3,7815882143
68,2020/02/18 3:50:02 PM EST,18,Male,Sophomore,Bioengineering,Yes,Oak Glen,Often,[Sustainability],No,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",3,jarevalo@umassd.edu
95,2020/02/18 4:11:22 PM EST,19,Female,Sophomore,Bioengineering,Yes,Oak Glen,Always,"[Sustainability, Ease of Access, Habit]",No,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",5,Avanvoorhis2@umassd.edu
62,2020/02/18 3:49:35 PM EST,19,Female,Sophomore,Bioengineering,No,No,Often,[Habit],Yes,No,No,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",4,kbernier1@umassd.edu
42,2020/02/18 3:42:22 PM EST,18,Female,Freshman,Biology,Yes,Maple,Always,"[Sustainability, Habit]",I didn't know there were any recycling rules!,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Glass Bott...",3,No
47,2020/02/18 3:43:19 PM EST,22,Female,Senior,Biology,Yes,Oak Glen,Always,[Sustainability],No,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Aluminum C...",5,Nenglish@umassd.edu
33,2020/02/18 3:39:48 PM EST,20,Female,Junior,Biology,Yes,Hickory,Always,[Sustainability],Yes,Yes,Yes,"[Paper, Cardboard, Plastic Bottles, Plastic Ba...",5,Email
19,2020/02/18 3:36:37 PM EST,22,Male,Junior,Civil Engineering,Yes,Willow,Always,[Sustainability],No,Yes,Yes,"[Paper, Plastic Bottles, Plastic Bags, Glass B...",5,Cell phone
5,2020/02/18 3:13:20 PM EST,20,Male,Junior,College of Business,No,No,Sometimes,[Ease of Access],Yes,No,Yes,"[Paper, Plastic Bottles, Aluminum Cans]",3,Jzheng@umassd.edu
